In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
'''
# ==================small size=================
# hyperparameters
batch_size = 8 # how many independent sequences will we process in parallel?
block_size = 512 # what is the maximum input context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embedding = 128 # 32 =  head_size = n_embedding / n_head
n_head = 8
n_layer = 20
dropout = 0.2
# ------------

#torch.manual_seed(1337)
'''

"\n# ==================small size=================\n# hyperparameters\nbatch_size = 8 # how many independent sequences will we process in parallel?\nblock_size = 512 # what is the maximum input context length for predictions?\nmax_iters = 5000\neval_interval = 500\nlearning_rate = 3e-4\ndevice = 'cuda' if torch.cuda.is_available() else 'cpu'\neval_iters = 200\nn_embedding = 128 # 32 =  head_size = n_embedding / n_head\nn_head = 8\nn_layer = 20\ndropout = 0.2\n# ------------\n\n#torch.manual_seed(1337)\n"

In [4]:
# hyperparameters
batch_size = 8 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embedding = 384
n_head = 24
n_layer = 12
dropout = 0.2
# ------------

cuda


In [5]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [6]:
'''
# Shakespeare
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    #print(text)
print(len(text))
'''

"\n# Shakespeare\nwith open('input.txt', 'r', encoding='utf-8') as f:\n    text = f.read()\n    #print(text)\nprint(len(text))\n"

In [7]:

# 7 habits of highly effective people
with open('7_habits.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    #print(text)
print(len(text))


649645


In [8]:
# here are all the unique characters that occur in this text
print(set(text))
print(list(set(text)))
print(sorted(list(set(text))))
print(len(sorted(list(set(text)))))
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(stoi['\n'])
print(encode(['A','B','C']))
print(decode(encode(['A','B','C'])))

{',', '|', 'R', 'y', '0', 'v', ' ', '#', 'M', 'J', 'L', 'A', 'C', '<', ':', '6', 'F', '&', 'i', '*', 'd', 'U', 'n', '}', '+', 'O', '!', '—', 'a', '□', 'm', 'Y', 'p', ')', '>', '^', 'u', '7', '2', ';', 'G', '”', '4', '9', '_', '.', '1', 'w', 'l', 'r', 'D', 'f', '’', 'T', "'", '\\', 'V', 'j', '=', 'c', 'E', '?', 'q', 'k', 'I', 'P', 'z', '¬', 't', 'H', 'K', 'Q', 'Z', 'N', '8', '"', 'S', 'b', 'g', '$', 'o', '‘', '(', '/', 'h', '3', '\n', 'X', '-', 'x', 'B', '[', 'e', '“', '5', 'W', 's'}
[',', '|', 'R', 'y', '0', 'v', ' ', '#', 'M', 'J', 'L', 'A', 'C', '<', ':', '6', 'F', '&', 'i', '*', 'd', 'U', 'n', '}', '+', 'O', '!', '—', 'a', '□', 'm', 'Y', 'p', ')', '>', '^', 'u', '7', '2', ';', 'G', '”', '4', '9', '_', '.', '1', 'w', 'l', 'r', 'D', 'f', '’', 'T', "'", '\\', 'V', 'j', '=', 'c', 'E', '?', 'q', 'k', 'I', 'P', 'z', '¬', 't', 'H', 'K', 'Q', 'Z', 'N', '8', '"', 'S', 'b', 'g', '$', 'o', '‘', '(', '/', 'h', '3', '\n', 'X', '-', 'x', 'B', '[', 'e', '“', '5', 'W', 's']
['\n', ' ', '!', '"', '#

In [8]:
# rich dad, poor dad
with open('rich_dad_poor_dad.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    #print(text)
print(len(text))

990416


In [9]:
# here are all the unique characters that occur in this text
print(set(text))
print(list(set(text)))
print(sorted(list(set(text))))
print(len(sorted(list(set(text)))))
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(stoi['\n'])
print(encode(['A','B','C']))
print(decode(encode(['A','B','C'])))

{'\\', 'w', ',', 'A', '#', '—', 'Z', '?', '5', "'", 'p', 'q', '(', 'm', 'M', '&', 'd', 'Q', 'b', '}', '^', '¬', '1', 'I', '+', '=', '2', 'E', '□', '$', 'f', '"', 'c', 'B', '|', 'N', ' ', ':', '’', '/', 'a', 'o', 'R', '0', 'g', 'K', 'W', 'l', '”', 'y', 'e', '6', '8', 't', 'n', '\n', 'S', 'j', 'z', '[', 'J', '!', 'i', 'L', 'F', '>', 'x', 'v', '.', '3', 'X', '*', ')', 'h', 'P', 's', '-', 'C', 'U', '9', 'D', 'H', 'G', 'Y', '<', '4', '“', '7', 'O', 'r', '‘', 'k', 'T', 'V', '_', 'u', ';'}
['\\', 'w', ',', 'A', '#', '—', 'Z', '?', '5', "'", 'p', 'q', '(', 'm', 'M', '&', 'd', 'Q', 'b', '}', '^', '¬', '1', 'I', '+', '=', '2', 'E', '□', '$', 'f', '"', 'c', 'B', '|', 'N', ' ', ':', '’', '/', 'a', 'o', 'R', '0', 'g', 'K', 'W', 'l', '”', 'y', 'e', '6', '8', 't', 'n', '\n', 'S', 'j', 'z', '[', 'J', '!', 'i', 'L', 'F', '>', 'x', 'v', '.', '3', 'X', '*', ')', 'h', 'P', 's', '-', 'C', 'U', '9', 'D', 'H', 'G', 'Y', '<', '4', '“', '7', 'O', 'r', '‘', 'k', 'T', 'V', '_', 'u', ';']
['\n', ' ', '!', '"', '#

In [9]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print(train_data[0:100])
print(text[0:100])
print(sorted(list(set(text))))


#block_size = 8
print(train_data[0:block_size+1])
print(text[0:block_size+1])


x = train_data[0:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[0:t+1]
  target = y[t]

tensor([51, 39, 36,  1, 50, 36, 53, 36, 45,  1, 39, 32, 33, 39, 50,  1, 46, 37,
         1, 39, 40, 38, 39, 43, 56,  1, 36, 37, 37, 36, 34, 51, 40, 53, 36,  1,
        47, 36, 46, 47, 43, 36,  1,  0,  0,  0, 33, 79, 62, 71, 64, 71, 81,  1,
        81, 76,  1, 86, 76, 82,  1, 81, 86, 39, 86, 39, 80, 82, 81,  1,  0,  0,
         0, 51, 39, 36,  1, 50, 36, 53, 36, 45,  1, 39, 32, 33, 40, 51, 50,  1,
        46, 37,  1, 39, 40, 38, 39, 43, 56,  1])
THE SEVEN HABHS OF HIGHLY EFFECTIVE PEOPLE 


Brajcjt to you tyHyHsut 


THE SEVEN HABITS OF HIGHLY 
['\n', ' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}

In [11]:
# data loading
#torch.manual_seed(1337)
#batch_size = 4
#block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

xb, yb = get_batch('train')
print("input:")
print(xb.shape)
print(xb)
print("target:")
print(yb.shape)
print(yb)

for b in range(batch_size):
  for t in range(block_size):
    context = xb[ b, 0:t+1 ]
    target = yb[ b, t ]
    #print(context,"->",target)
    
    

input:
torch.Size([8, 128])
tensor([[81, 79, 70,  ...,  0,  0, 69],
        [82, 73,  1,  ..., 81, 66, 79],
        [67, 66, 62,  ..., 81, 66, 62],
        ...,
        [82,  1, 81,  ..., 77, 80,  1],
        [76, 67, 66,  ..., 69, 81,  1],
        [ 1, 65, 70,  ..., 75, 66, 84]], device='cuda:0')
target:
torch.Size([8, 128])
tensor([[79, 70, 66,  ...,  0, 69, 81],
        [73,  1, 67,  ..., 66, 79,  1],
        [66, 62, 79,  ..., 66, 62, 64],
        ...,
        [ 1, 81, 69,  ..., 80,  1, 62],
        [67, 66, 80,  ..., 81,  1, 81],
        [65, 70, 80,  ..., 66, 84, 12]], device='cuda:0')


In [12]:
class GateLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GateLayer, self).__init__()
        
        self.linear = nn.Linear(input_dim, n_embedding)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, encoder_output):
        # Compute gating vector
        gating_vector = self.linear(encoder_output)
        gating_vector = self.sigmoid(gating_vector)
        # Apply gating vector to encoder output
        gated_encoder_output = encoder_output * gating_vector
        
        return gated_encoder_output



class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key  = nn.Linear(n_embedding, head_size, bias=False)
        self.query = nn.Linear(n_embedding, head_size, bias=False)
        self.value = nn.Linear(n_embedding, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
        self.gate = GateLayer( n_embedding, head_size )


    def forward(self, x):
        B,T,C = x.shape
        #print("====x=====")
        #print(x.shape)
        #x = self.gate(x)

        k = self.key(x)   # (B,T,C)
        #print(k.shape)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        attention_part = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        attention_part = attention_part.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        attention_part = F.softmax(attention_part, dim=-1) # (B, T, T)
        attention_part = self.dropout(attention_part)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = attention_part @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.linear = nn.Linear(n_embedding, n_embedding)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.linear(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embedding):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embedding, 4 * n_embedding),
            nn.ReLU(),
            nn.Linear(4 * n_embedding, n_embedding),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embedding, n_head):
        # n_embedding: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embedding // n_head
        self.multi_head_attention = MultiHeadAttention(n_head, head_size)
        self.feed_forward = FeedFoward(n_embedding)
        self.layer_norm_1 = nn.LayerNorm(n_embedding)
        self.layer_norm_2 = nn.LayerNorm(n_embedding)

    def forward(self, x):
        x = x + self.multi_head_attention(self.layer_norm_1(x))
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

# super simple bigram model
class GPT2(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_into_embedding = nn.Embedding(vocab_size, n_embedding)
        self.position_embedding_table = nn.Embedding(block_size, n_embedding)
        
        #Input: (∗), IntTensor or LongTensor of arbitrary shape containing the indices to extract
        #Output: (∗,H), where * is the input shape and H=embedding_dim
        
        
        self.blocks = nn.Sequential(*[Block(n_embedding, n_head=n_head) for _ in range(n_layer)])
        self.layer_norm = nn.LayerNorm(n_embedding) # final layer norm
        #self.ffwd = FeedFoward(n_embedding)
        #self.sa_head = MultiHeadAttention(4,n_embedding//4)
        self.linear_head = nn.Linear(n_embedding, vocab_size)

    def forward(self, id_number_of_vector_x, targets=None):#target (B,T)
        #B,T =id_number_of_vector_x.shape
        #tok_embed = self.token_into_embedding(id_number_of_vector_x) #(B,T,C) (batch,Time,Channel)
        #logits = self.lm_head(tok_embed) #(B,T,vocab_size)

        B, T = id_number_of_vector_x.shape

        # id_number_of_vector_x and targets are both (B,T) tensor of integers
        #tok_emb = self.token_into_embedding(id_number_of_vector_x) # (B,T,C)
        #pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        #x = tok_emb + pos_emb # (B,T,C) + pos_emb across the batch
        #x = self.blocks(x) # (B,T,C)
        #x = self.ln_f(x) # (B,T,C)
        
        #x = self.sa_head(x) # (B,T,vocab_size)
        #logits = self.lm_head(x) # (B,T,vocab_size)

        # id_number_of_vector_x and targets are both (B,T) tensor of integers
        tok_emb = self.token_into_embedding(id_number_of_vector_x) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        #x = tok_emb + pos_emb # (B,T,C)
        #x = self.blocks(x) # (B,T,C)
        #x = self.ln_f(x) # (B,T,C)
        #x = self.sa_head(x) # (B,T,vocab_size)
        #x = self.ffwd(x)
        #logits = self.lm_head(x) # (B,T,vocab_size)

        x = tok_emb + pos_emb # (B,T,C) + pos_emb across the batch
        x = self.blocks(x) # (B,T,C)
        x = self.layer_norm(x) # (B,T,C)
        
        #x = self.sa_head(x) # (B,T,vocab_size)
        logits = self.linear_head(x) # (B,T,vocab_size)

        if targets == None:
          loss = None
        else:
          B, T, C = logits.shape # logit(p) = ln( p / ( 1 - p ) )
          logits = logits.view(B*T,C)
          targets = targets.view(B*T)
          loss = F.cross_entropy(logits,targets) # H( P, Q ) = -0.9 * log( 0.8 ) - 0.1 * log( 0.2 ) = 0.311, the lower the better matching
        return logits, loss

    def generate(self, id_number_of_vector_x, max_new_tokens):
        # id_number_of_vector_x is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop id_number_of_vector_x to the last block_size tokens
            id_number_of_vector_x_cut = id_number_of_vector_x[:, -block_size:]
            # get the predictions
            logits, loss = self(id_number_of_vector_x_cut)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            id_number_of_vector_x_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            id_number_of_vector_x = torch.cat((id_number_of_vector_x, id_number_of_vector_x_next), dim=1) # (B, T+1)
        return id_number_of_vector_x


In [13]:
'''
model.load_state_dict(torch.load("./model"))
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
'''

model = GPT2()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

logits, loss = m(xb,yb)
print(logits.shape,loss) # -ln(1/65) = 4.174387269896

63.982177 M parameters
torch.Size([1024, 97]) tensor(4.7217, device='cuda:0', grad_fn=<NllLossBackward0>)


In [14]:

model = GPT2()
#model.load_state_dict(torch.load("./GPT2_Shakespeare"))
#model.load_state_dict(torch.load("./GPT2_rich_dad_poor_dad"))
model.load_state_dict(torch.load("./GPT2_rich_dad_poor_dad_Fine-tuning_with_Custom_Datasets"))

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

id_number_of_vector_x = torch.zeros((1,1), dtype=torch.long)
print(id_number_of_vector_x)
print(torch.tensor([[32]]).to(device))
#print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=10)[0].tolist()))

print(decode(m.generate(torch.tensor([[32]]).to(device), max_new_tokens=100)[0].tolist()))


63.982177 M parameters
tensor([[0]])
tensor([[32]], device='cuda:0')
A Personal M inds 

You wearehour is greater income; and hence the 
fame of retirement. Paradigm Shif


In [15]:
# not working because of the small size of the LLM
text = "This is what world famous speaker and author Anthony Robbins says about Robert’s work. "
context = torch.tensor( [ encode(text) ], dtype=torch.long, device=device )
print(decode(m.generate( context, max_new_tokens=100 )[0].tolist()))

This is what world famous speaker and author Anthony Robbins says about Robert’s work. 

The reality of change is in the entire center’s common. 
We sensee of empathic affects the other a


In [16]:
# not working because of the small size of the LLM
text = "how about the PC, and money, how do you think about the PC and learning"
context = torch.tensor( [ encode(text) ], dtype=torch.long, device=device )
print(decode(m.generate( context, max_new_tokens=1000 )[0].tolist()))

how about the PC, and money, how do you think about the PC and learning 

"Dady hard more than just going along with your friends and you did not crease?" 

Because those a character empowers us. With correct principles, with panic hand bitterms and 
are important for malfilure in management. Balances in which you are freedom. Try to accomplish these principles, 
knowledge and there are more in the second creation, no appreciation; win someone else's words have enough 
creating. Put another good reeply. N eighborce to feedback with a “best-selling author,”</p> <div 
class=’story_text’>" And how I do went because, we do not respondence, and sittle two training for 
a conspencial mental while Was So on the big -- selling script, and was recipited with old patern. 

"Who're the gold meminatry was happening in our interactions with others, what these popular habits 
are habits. 

My habits of real estate and marriage are so copeaning other untity. 
Desp everything I have never communicatio

In [19]:
# not working because of the small size of the LLM
text = "how about the Production Capability, and money, how do you think about the Porduction Capability and learning"
context = torch.tensor( [ encode(text) ], dtype=torch.long, device=device )
print(decode(m.generate( context, max_new_tokens=1000 )[0].tolist()))

how about the Production Capability, and money, how do you think about the Porduction Capability and learning 
communication exposite skills that have enorcept others. In cases it, simply because they find 
a Power or Psyptent Ce, sitting a collecting child good way, a decision that wanted to understand 
either or forced to inside his energy. 

Spent the future coffee interned opening to enjuy Mike. And more intimate interuth to her. 

There before we expensive wikly on the Seven Habits is, too many real self-reliant with our own 
confront is evaparane, ulevision. There are times to go through these when they 
were subject. 

No our schedule to think, on the other hand, performance the divi difference in an attempt of 
principles and disciplineess. Many five major income up soundies to 
work to master than money. They’re too our speaker -1, So The jobs one and terribure show hard from 
rich. Don’t get on my own company can career. Instead of the United 
States Corp mode to stay "Just l

In [20]:
# not working because of the small size of the LLM
text = "how to make money"
context = torch.tensor( [ encode(text) ], dtype=torch.long, device=device )
print(decode(m.generate( context, max_new_tokens=1000 )[0].tolist()))

how to make money is to inice ut of school to school because pulled and no 
supervise methods. Dearn freedoms in the reality that changes we 
did not come to love. 


Suppose time I say to many to manipuls into the marriages considerations of love to significantly 
describe about those rich ship that most; they are the most, looking at their lives, their next only 
tax door because on their taxes.” Most people are interest of us around. Most peoplesis are 
not mire people are more objective and maintance in their work. 

Going on the main juni insigh school is going to be a coming in the perspective wealth. 

We inside the earlier man, revenues to give picture them. Teaning sidestily new captured into their 
incomes paking and give in their spirit because you go to scope to where yonce you make the time into 
fact, and I went to schedule out of that obne, our boss!" 

We went into a ti me al
Our envision of considerating coum from than the problem. 

I'm not intrinsic society. Indifyin

In [24]:
print(decode(m.generate(torch.tensor([[32]]).to(device), max_new_tokens=100)[0].tolist()))

Another 
day, whicholocing work against the glocum. They might decide to tell you the same this tyle 


In [22]:
print(dir(m.blocks[0].multi_head_attention.heads[0].key))
print(m.blocks[0].multi_head_attention.heads[0].key.type)
print(m.blocks[0].multi_head_attention.heads[0].key.in_features)
print(m.blocks[0].multi_head_attention.heads[0].key.out_features)
print(m.blocks[0].multi_head_attention.heads[0].key.weight.shape)
print(m.blocks[0].multi_head_attention.heads[0].key.weight[0].shape)
#print(m.blocks[0].multi_head_attention.heads[0].key.weight[0])
print(dir(m.token_into_embedding))
print(dir(m.token_into_embedding.weight))
print(m.token_into_embedding.weight.shape)
print(m.token_into_embedding.weight[96])

#Input: (∗), IntTensor or LongTensor of arbitrary shape containing the indices to extract

#Output: (∗,H), where * is the input shape and =embedding_dim H=embedding_dim

'''
['T_destination', '__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'bias', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'half', 'in_features', 'load_state_dict', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'out_features', 'parameters', 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_parameter', 'requires_grad_', 'reset_parameters', 'share_memory', 'state_dict', 'to', 'train', 'training', 'type', 'weight', 'zero_grad']
<bound method Module.type of Linear(in_features=384, out_features=16, bias=False)>
384
16
torch.Size([16, 384])
torch.Size([384])
['T_destination', '__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dump_patches', 'embedding_dim', 'eval', 'extra_repr', 'float', 'forward', 'from_pretrained', 'half', 'load_state_dict', 'max_norm', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'norm_type', 'num_embeddings', 'padding_idx', 'parameters', 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_parameter', 'requires_grad_', 'reset_parameters', 'scale_grad_by_freq', 'share_memory', 'sparse', 'state_dict', 'to', 'train', 'training', 'type', 'weight', 'zero_grad']
['T', '__abs__', '__add__', '__and__', '__array__', '__array_priority__', '__array_wrap__', '__bool__', '__class__', '__complex__', '__contains__', '__cuda_array_interface__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__iand__', '__idiv__', '__ifloordiv__', '__ilshift__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pow__', '__radd__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rfloordiv__', '__rmul__', '__rpow__', '__rshift__', '__rsub__', '__rtruediv__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__torch_function__', '__truediv__', '__weakref__', '__xor__', '_backward_hooks', '_base', '_cdata', '_coalesced_', '_dimI', '_dimV', '_grad', '_grad_fn', '_indices', '_is_view', '_make_subclass', '_nnz', '_update_names', '_values', '_version', 'abs', 'abs_', 'absolute', 'absolute_', 'acos', 'acos_', 'acosh', 'acosh_', 'add', 'add_', 'addbmm', 'addbmm_', 'addcdiv', 'addcdiv_', 'addcmul', 'addcmul_', 'addmm', 'addmm_', 'addmv', 'addmv_', 'addr', 'addr_', 'align_as', 'align_to', 'all', 'allclose', 'amax', 'amin', 'angle', 'any', 'apply_', 'arccos', 'arccos_', 'arccosh', 'arccosh_', 'arcsin', 'arcsin_', 'arcsinh', 'arcsinh_', 'arctan', 'arctan_', 'arctanh', 'arctanh_', 'argmax', 'argmin', 'argsort', 'as_strided', 'as_strided_', 'as_subclass', 'asin', 'asin_', 'asinh', 'asinh_', 'atan', 'atan2', 'atan2_', 'atan_', 'atanh', 'atanh_', 'backward', 'baddbmm', 'baddbmm_', 'bernoulli', 'bernoulli_', 'bfloat16', 'bincount', 'bitwise_and', 'bitwise_and_', 'bitwise_not', 'bitwise_not_', 'bitwise_or', 'bitwise_or_', 'bitwise_xor', 'bitwise_xor_', 'bmm', 'bool', 'byte', 'cauchy_', 'ceil', 'ceil_', 'char', 'cholesky', 'cholesky_inverse', 'cholesky_solve', 'chunk', 'clamp', 'clamp_', 'clamp_max', 'clamp_max_', 'clamp_min', 'clamp_min_', 'clip', 'clip_', 'clone', 'coalesce', 'conj', 'contiguous', 'copy_', 'cos', 'cos_', 'cosh', 'cosh_', 'count_nonzero', 'cpu', 'cross', 'cuda', 'cummax', 'cummin', 'cumprod', 'cumsum', 'data', 'data_ptr', 'deg2rad', 'deg2rad_', 'dense_dim', 'dequantize', 'det', 'detach', 'detach_', 'device', 'diag', 'diag_embed', 'diagflat', 'diagonal', 'digamma', 'digamma_', 'dim', 'dist', 'div', 'div_', 'divide', 'divide_', 'dot', 'double', 'dtype', 'eig', 'element_size', 'eq', 'eq_', 'equal', 'erf', 'erf_', 'erfc', 'erfc_', 'erfinv', 'erfinv_', 'exp', 'exp2', 'exp2_', 'exp_', 'expand', 'expand_as', 'expm1', 'expm1_', 'exponential_', 'fft', 'fill_', 'fill_diagonal_', 'fix', 'fix_', 'flatten', 'flip', 'fliplr', 'flipud', 'float', 'floor', 'floor_', 'floor_divide', 'floor_divide_', 'fmod', 'fmod_', 'frac', 'frac_', 'gather', 'gcd', 'gcd_', 'ge', 'ge_', 'geometric_', 'geqrf', 'ger', 'get_device', 'grad', 'grad_fn', 'greater', 'greater_', 'greater_equal', 'greater_equal_', 'gt', 'gt_', 'half', 'hardshrink', 'has_names', 'heaviside', 'heaviside_', 'histc', 'hypot', 'hypot_', 'i0', 'i0_', 'ifft', 'imag', 'index_add', 'index_add_', 'index_copy', 'index_copy_', 'index_fill', 'index_fill_', 'index_put', 'index_put_', 'index_select', 'indices', 'int', 'int_repr', 'inverse', 'irfft', 'is_coalesced', 'is_complex', 'is_contiguous', 'is_cuda', 'is_distributed', 'is_floating_point', 'is_leaf', 'is_meta', 'is_mkldnn', 'is_nonzero', 'is_pinned', 'is_quantized', 'is_same_size', 'is_set_to', 'is_shared', 'is_signed', 'is_sparse', 'isclose', 'isfinite', 'isinf', 'isnan', 'isneginf', 'isposinf', 'isreal', 'istft', 'item', 'kthvalue', 'layout', 'lcm', 'lcm_', 'le', 'le_', 'lerp', 'lerp_', 'less', 'less_', 'less_equal', 'less_equal_', 'lgamma', 'lgamma_', 'log', 'log10', 'log10_', 'log1p', 'log1p_', 'log2', 'log2_', 'log_', 'log_normal_', 'log_softmax', 'logaddexp', 'logaddexp2', 'logcumsumexp', 'logdet', 'logical_and', 'logical_and_', 'logical_not', 'logical_not_', 'logical_or', 'logical_or_', 'logical_xor', 'logical_xor_', 'logit', 'logit_', 'logsumexp', 'long', 'lstsq', 'lt', 'lt_', 'lu', 'lu_solve', 'map2_', 'map_', 'masked_fill', 'masked_fill_', 'masked_scatter', 'masked_scatter_', 'masked_select', 'matmul', 'matrix_exp', 'matrix_power', 'max', 'maximum', 'mean', 'median', 'min', 'minimum', 'mm', 'mode', 'movedim', 'mul', 'mul_', 'multinomial', 'multiply', 'multiply_', 'mv', 'mvlgamma', 'mvlgamma_', 'name', 'names', 'nanquantile', 'nansum', 'narrow', 'narrow_copy', 'ndim', 'ndimension', 'ne', 'ne_', 'neg', 'neg_', 'negative', 'negative_', 'nelement', 'new', 'new_empty', 'new_full', 'new_ones', 'new_tensor', 'new_zeros', 'nextafter', 'nextafter_', 'nonzero', 'norm', 'normal_', 'not_equal', 'not_equal_', 'numel', 'numpy', 'orgqr', 'ormqr', 'outer', 'output_nr', 'permute', 'pin_memory', 'pinverse', 'polygamma', 'polygamma_', 'pow', 'pow_', 'prelu', 'prod', 'put_', 'q_per_channel_axis', 'q_per_channel_scales', 'q_per_channel_zero_points', 'q_scale', 'q_zero_point', 'qr', 'qscheme', 'quantile', 'rad2deg', 'rad2deg_', 'random_', 'real', 'reciprocal', 'reciprocal_', 'record_stream', 'refine_names', 'register_hook', 'reinforce', 'relu', 'relu_', 'remainder', 'remainder_', 'rename', 'rename_', 'renorm', 'renorm_', 'repeat', 'repeat_interleave', 'requires_grad', 'requires_grad_', 'reshape', 'reshape_as', 'resize', 'resize_', 'resize_as', 'resize_as_', 'retain_grad', 'rfft', 'roll', 'rot90', 'round', 'round_', 'rsqrt', 'rsqrt_', 'scatter', 'scatter_', 'scatter_add', 'scatter_add_', 'select', 'set_', 'sgn', 'sgn_', 'shape', 'share_memory_', 'short', 'sigmoid', 'sigmoid_', 'sign', 'sign_', 'signbit', 'sin', 'sin_', 'sinh', 'sinh_', 'size', 'slogdet', 'smm', 'softmax', 'solve', 'sort', 'sparse_dim', 'sparse_mask', 'sparse_resize_', 'sparse_resize_and_clear_', 'split', 'split_with_sizes', 'sqrt', 'sqrt_', 'square', 'square_', 'squeeze', 'squeeze_', 'sspaddmm', 'std', 'stft', 'storage', 'storage_offset', 'storage_type', 'stride', 'sub', 'sub_', 'subtract', 'subtract_', 'sum', 'sum_to_size', 'svd', 'symeig', 't', 't_', 'take', 'tan', 'tan_', 'tanh', 'tanh_', 'to', 'to_dense', 'to_mkldnn', 'to_sparse', 'tolist', 'topk', 'trace', 'transpose', 'transpose_', 'triangular_solve', 'tril', 'tril_', 'triu', 'triu_', 'true_divide', 'true_divide_', 'trunc', 'trunc_', 'type', 'type_as', 'unbind', 'unflatten', 'unfold', 'uniform_', 'unique', 'unique_consecutive', 'unsafe_chunk', 'unsafe_split', 'unsafe_split_with_sizes', 'unsqueeze', 'unsqueeze_', 'values', 'var', 'vdot', 'view', 'view_as', 'where', 'zero_']
torch.Size([97, 384])
tensor([-1.3664e-01, -4.0590e-01, -2.2219e-01,  5.3633e-01,  1.3847e+00,
         6.0940e-01,  1.0424e+00,  1.1116e+00, -2.6177e-01, -1.1758e-01,
        -8.0097e-01,  3.9556e-01,  1.8125e-01,  4.5105e-01,  2.7599e-01,
         5.1245e-01,  5.1657e-01,  1.4031e+00, -6.5616e-01,  3.6901e-01,
         7.3739e-01, -2.9283e-01, -1.2868e+00, -7.9836e-01, -2.4348e-01,
        -4.3813e-01,  3.0676e-01,  8.2756e-01, -1.9864e-01, -4.6908e-01,
         4.0065e-01,  2.0083e+00, -5.0702e-01,  7.8803e-01, -9.9201e-01,
         2.9955e-01, -1.2661e-01,  6.3483e-02,  5.6666e-01,  5.4232e-01,
         9.1668e-01, -1.0347e-01,  1.1669e+00,  1.8649e+00,  5.9467e-02,
         8.1702e-01, -4.9200e-01,  8.2695e-01, -9.3656e-01, -7.9871e-01,
         1.0738e+00, -4.7494e-01, -7.2439e-01, -2.2799e-01,  3.0416e-01,
         1.1063e+00, -4.2677e-01,  6.1318e-01,  1.3603e+00, -1.1408e+00,
        -7.3598e-01, -7.5957e-01, -6.9394e-02, -2.5286e-01,  4.9853e-01,
         4.6131e-01, -6.1774e-01,  7.9585e-01,  4.2797e-01, -1.7809e-01,
        -1.9401e+00, -5.5100e-01,  2.6329e-01,  1.1866e+00, -5.6975e-01,
        -1.1178e-01, -5.2068e-02,  1.3399e+00,  1.2692e-01, -1.9369e+00,
        -1.5599e-01, -3.0305e-01, -2.8277e+00,  1.5071e-02,  2.3361e+00,
        -3.8259e-01, -3.2923e-01, -5.6894e-01,  7.5279e-01, -1.4578e+00,
        -1.9438e+00, -2.3516e+00, -4.1028e-02, -3.2798e-02, -5.7765e-01,
        -3.2116e-01, -2.5557e-01,  1.0580e-01,  4.3000e-01,  3.2293e-01,
         2.7354e-01,  6.9304e-01,  1.8769e+00, -3.9564e-01, -1.3200e+00,
         2.2713e-01,  4.8751e-01, -1.3580e+00, -3.8609e-01, -1.0239e+00,
        -1.4512e-01, -2.3120e-01,  4.3180e-01, -5.5675e-01, -7.9909e-01,
        -7.5038e-01,  8.4322e-01, -3.1666e-01,  3.4654e-01,  7.7588e-01,
         3.8141e-01, -2.0563e-01,  3.1469e-01, -8.0654e-01,  4.7067e-01,
         1.0281e+00, -4.8603e-01, -2.9031e+00,  4.1955e-01,  1.6954e+00,
        -9.4966e-01, -2.0211e-01, -5.6915e-02, -7.5391e-01,  5.8532e-01,
         1.3990e+00, -8.8408e-02,  1.9064e-01,  7.5883e-01,  2.4874e-03,
        -2.5647e-02,  2.1273e-01,  1.1395e+00, -5.0308e-01,  8.5667e-01,
         8.8261e-01,  8.0532e-01, -8.7556e-01, -1.4758e-01, -1.4905e-01,
         5.1957e-02, -9.1003e-01,  6.8148e-01, -5.4598e-01,  5.3370e-01,
         3.8573e-01,  3.3385e-02,  1.6973e-01, -1.8141e-01, -7.0944e-01,
        -1.2175e+00, -1.1221e+00,  1.2512e-01,  7.7090e-01, -5.4690e-01,
         9.2581e-01, -1.1107e-01,  7.5230e-01,  1.5444e+00, -1.1679e+00,
        -5.8597e-01,  1.0054e+00,  5.6257e-02,  4.0462e-01,  3.9183e-01,
         6.4785e-01,  1.2511e-02,  7.7656e-01,  1.0831e+00, -7.0587e-01,
        -1.8596e-01, -1.8027e+00, -6.2220e-01,  1.2311e+00, -1.7799e+00,
        -1.0808e+00, -6.7453e-01, -1.1141e-01,  8.4733e-02,  1.9948e-02,
        -4.0087e-01,  3.3899e-01,  8.1522e-01,  1.5621e+00, -7.0142e-01,
        -4.8223e-01, -6.6697e-01, -1.8925e-01,  1.1880e+00, -1.2273e+00,
         8.9999e-01,  5.9492e-01,  8.1438e-01,  1.5666e-02, -8.8533e-01,
         4.1776e-01, -6.4685e-01,  4.5221e-01,  2.5957e-02,  3.9510e-01,
        -8.5163e-01,  8.2882e-01,  9.5060e-01, -4.4255e-01, -9.2770e-01,
        -1.5975e+00,  2.9371e-01, -1.2315e+00, -8.7343e-01,  8.9428e-02,
         1.7647e-01, -7.4679e-01,  2.8629e+00, -4.5990e-01, -1.6549e-01,
         1.1021e+00, -1.6458e+00, -4.1318e-02,  2.1957e+00,  1.1366e+00,
         1.5442e-01, -1.3475e+00, -1.0809e+00,  1.2546e+00, -8.4330e-01,
         4.5855e-01, -4.8445e-02, -6.4026e-01, -2.1022e+00, -1.0389e-01,
        -1.2566e+00,  1.0140e+00,  4.5058e-01, -6.1631e-01, -8.6026e-01,
         1.2391e+00, -6.6977e-01,  7.2814e-01,  6.4342e-01, -7.0971e-01,
        -9.4501e-01,  7.4483e-01,  2.8113e-01,  1.3425e+00, -1.0013e+00,
        -9.8773e-01,  1.6508e+00,  1.0094e+00, -1.3246e+00,  4.7838e-02,
        -5.5132e-01,  1.8356e+00,  4.9750e-01,  8.4641e-01, -6.9083e-01,
        -4.2841e-01, -4.1163e-01,  6.7305e-01, -7.4259e-01, -3.8063e-01,
         1.4060e-01, -1.2704e-01, -4.7451e-01,  2.1874e-01,  1.1581e-01,
         4.9308e-01,  1.5480e-01,  4.7753e-01, -1.5080e+00,  1.0588e+00,
        -1.9669e+00,  2.1036e+00,  7.2395e-01, -6.3684e-01, -7.8241e-01,
         6.4944e-01,  2.8066e-03,  1.3208e+00, -1.5772e+00,  1.0723e+00,
        -8.2203e-01,  1.4328e+00, -2.3067e-01,  7.0783e-01, -4.4902e-01,
        -1.3776e+00, -4.4486e-01, -6.3538e-01,  2.1044e-01, -1.1066e+00,
         1.2276e+00, -1.0044e+00,  7.8033e-01, -1.0372e-01, -7.7795e-01,
        -8.6399e-01,  1.1657e+00, -7.7011e-01,  6.8821e-01, -2.3464e+00,
        -1.0903e+00,  1.0466e+00, -2.4827e-01,  8.8626e-01, -1.4259e+00,
        -1.1011e+00, -5.7526e-01,  8.8903e-01,  1.0949e+00, -8.0354e-01,
        -3.5087e-01, -5.7875e-02,  6.7720e-01, -8.5290e-02, -6.5205e-02,
        -3.7540e-01, -5.5354e-01, -2.8629e-01,  3.4709e-01, -8.3628e-01,
        -1.1885e+00,  1.1538e+00,  2.3160e-01, -1.4859e-01, -1.4890e+00,
         5.2647e-01,  6.6733e-01, -8.2302e-01,  3.9545e-01, -2.9798e-01,
        -7.5579e-01,  7.2871e-01, -3.2641e-01, -9.1883e-01, -4.0755e-01,
         1.5744e-01, -5.8806e-02, -6.9044e-01,  4.2294e-01, -1.4729e+00,
        -4.8012e-01, -1.0481e+00, -4.4776e-01, -2.7175e-01,  2.2290e-02,
        -2.1795e+00,  1.0271e+00, -7.6579e-01,  3.9064e-01,  7.7011e-01,
        -6.5713e-01,  2.2870e-01,  6.1872e-01, -2.0098e-01,  2.5962e-02,
        -1.5894e+00, -8.6020e-01, -1.8814e+00, -7.1663e-02,  9.1778e-01,
         9.2523e-02, -1.2062e+00, -1.1386e+00, -8.6978e-01,  1.0440e+00,
        -2.7735e+00,  1.2994e+00,  1.2769e+00, -2.4975e-01, -1.2188e+00,
        -5.0826e-01,  8.4310e-01,  8.4368e-01, -7.9602e-02], device='cuda:0',
       grad_fn=<SelectBackward>)
       
tensor([ 2.1269e+00, -2.0395e+00,  2.5961e-01,  9.5541e-02,  9.1339e-01,

with 7 habits
tensor([ 2.1269e+00, -2.0395e+00,  2.5961e-01,  9.5541e-02,  9.1339e-01,
       
'''


['T_destination', '__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'bias', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward',

"\n['T_destination', '__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'bias', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forwar

In [ ]:
'''
# low 6.689373 M parameters

# hyperparameters
batch_size = 8 # how many independent sequences will we process in parallel?
block_size = 512 # what is the maximum input context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embedding = 128 # 32 =  head_size = n_embedding / n_head
n_head = 8
n_layer = 20
dropout = 0.2
# ------------

#torch.manual_seed(1337)
'''

In [21]:
# create a PyTorch optimizer
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
optimizer = torch.optim.AdamW(m.parameters(), lr=17e-5)

for iter in range(50000):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("====result_loss====")
print(loss.item())

#torch.save(m.state_dict(), "./GPT2_Shakespeare")
#torch.save(m.state_dict(), "./GPT2_7_habits")
torch.save(m.state_dict(), "./GPT2_rich_dad_poor_dad_Fine-tuning_with_Custom_Datasets")
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long,device=device), max_new_tokens=1000)[0].tolist()))

'''
#torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key  = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)

k = key(x) #(B,T,16)
q = query(x) #(B,T,16)
v = value(x)

wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) => (B,T,T)
tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)

output = wei @ v
output.shape
wei[0]



torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1)
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8,dim=-1)

'''


63.982177 M parameters
step 0: train loss 0.5041, val loss 1.5775
step 500: train loss 0.4960, val loss 1.5948
step 1000: train loss 0.4876, val loss 1.6042
step 1500: train loss 0.4814, val loss 1.5957
step 2000: train loss 0.4724, val loss 1.6051
step 2500: train loss 0.4674, val loss 1.5999
step 3000: train loss 0.4651, val loss 1.6204
step 3500: train loss 0.4581, val loss 1.6069
step 4000: train loss 0.4572, val loss 1.6743
step 4500: train loss 0.4436, val loss 1.6507
step 5000: train loss 0.4426, val loss 1.6450
step 5500: train loss 0.4402, val loss 1.6680
step 6000: train loss 0.4293, val loss 1.6619
step 6500: train loss 0.4234, val loss 1.6557
step 7000: train loss 0.4204, val loss 1.7017
step 7500: train loss 0.4148, val loss 1.7061
step 8000: train loss 0.4148, val loss 1.7196
step 8500: train loss 0.4082, val loss 1.6866
step 9000: train loss 0.4068, val loss 1.7168
step 9500: train loss 0.3944, val loss 1.7250
step 10000: train loss 0.3875, val loss 1.7125
step 10500: tr

"\n#torch.manual_seed(1337)\nB,T,C = 4,8,32\nx = torch.randn(B,T,C)\n\nhead_size = 16\nkey  = nn.Linear(C,head_size,bias=False)\nquery = nn.Linear(C,head_size,bias=False)\nvalue = nn.Linear(C,head_size,bias=False)\n\nk = key(x) #(B,T,16)\nq = query(x) #(B,T,16)\nv = value(x)\n\nwei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) => (B,T,T)\ntril = torch.tril(torch.ones(T,T))\n#wei = torch.zeros((T,T))\nwei = wei.masked_fill(tril==0,float('-inf'))\nwei = F.softmax(wei,dim=-1)\n\noutput = wei @ v\noutput.shape\nwei[0]\n\n\n\ntorch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1)\ntorch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8,dim=-1)\n\n"

In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long,device=device), max_new_tokens=1000)[0].tolist()))

In [ ]:
text = "How to get rich fast"
context = torch.tensor( [ encode(text) ], dtype=torch.long, device=device )
print(decode(m.generate( context, max_new_tokens=1000 )[0].tolist()))

In [15]:
text = "How to get rich fast"
context = torch.tensor( [ encode(text) ], dtype=torch.long, device=device )
print(decode(m.generate( context, max_new_tokens=1000 )[0].tolist()))

How to get rich faster, I suggested that the interpretation of what my highest was born and 
pause the problem. My flow dad I could not have the credit with the bank to buy another house, since he was 
almost skills. They need $7,000 at the difference between way. 

The extent of the problem was the 60 years of the three generally thried with his 
6 country went to enjoy in the 303es. 

"Look, there is days always gone?" I asked myself goals to a start they listening nothing to manage. 

Rich dad guestors for a pattern. But I was the other and his life went to riding good grades, and they not only to 
line out. In other words, most people because they would encourage to money to go because they 
want to love them with the most pain forward. They were created, dealing long 
his neither words. 

The wall today, there were antimal panic is most important. Give memore than our 
moods and the weaknesses of other people. They think simply genuinely 
happy for the masses being getting some an

In [16]:
# not working because of the small size of the LLM
text = "This is what world famous speaker and author Anthony Robbins says about Robert’s work. "
context = torch.tensor( [ encode(text) ], dtype=torch.long, device=device )
print(decode(m.generate( context, max_new_tokens=10000 )[0].tolist()))

This is what world famous speaker and author Anthony Robbins says about Robert’s work. 

“Robert Kiyosaki’s work in education he’s not payer,” I said with a sly grin. 

“Look, I guess for you to have to be job, better smarter. Then you are the money, instead of being an 
little changes that there are no recal today. There are more and the things 

Power 

"Isea, we walk you to think school. You something something kind of the second doing to these 
committed-the boy." 

"Id yes?" 

"No, not really. Dad, I don't turn you with me to the schoolists trainked in the happy marriage," I say. 
"Sound you panice the game off the things first go 
net into opening. You cannot pretend your standard of listening in your life and to be 
influenced. 

When you listen, you have a profession and will be wealthy promoted law. But, as a result, 
communication is problemed, correctly inspired or over and interpreted in loyalty to old these 
methods may be limited in purpose and even body the subtial force

In [18]:
# 1. linear neural network learning the price, w is the price, x is the amount, the loss is the total price
what is the GPT learning
w is [not known, maybe key, query, value], x is the word chain, the loss is the next char of the word chain